In [23]:
from pathlib import Path
from easse import sari, bleu, fkgl, bertscore, quality_estimation # samsa fails dep: tupa

import json
import numpy as np

In [2]:
def read_split_lines(file, n=3):
    src_sents, refs_sents = [], []
    with open(file, 'r', encoding='utf8') as f:
        for i, line in enumerate(f):
            if i == n:
                break
            line = line.strip().split('\t')
            if len(line) > 1:
                src_sents.append(line[0])
                refs_sents.append(line[1:])
            else:
                src_sents.append(line)

    refs_sents = list(map(list, [*zip(*refs_sents)]))
    
    return src_sents, refs_sents

src_sents, refs_sents = read_split_lines('/srv/scratch6/kew/ats/data/en/aligned/newsela_manual_v0_v1_test.tsv')


In [3]:
src_sents

['SEATTLE — The Chinook tribes greeted Lewis and Clark when the explorers arrived at the Lower Columbia River in 1805.',
 'The tribes helped members of the expedition through the winter, bringing them food and assisting with navigation.',
 'More than 200 years after that first contact, the five tribes that now make up the Chinook Indian Nation have a constitution, a tribal council and annual meetings.']

In [4]:
refs_sents

[['SEATTLE — The Chinook tribes greeted Lewis and Clark when the explorers arrived at the Lower Columbia River in 1805.',
  'The tribes helped members of the expedition through the winter, bringing them food and assisting with navigation.',
  'More than 200 years after that first contact, the five tribes that now make up the Chinook Indian Nation have a constitution, a tribal council and annual meetings.']]

In [5]:
hyp_sents = [
    'Chinook tribes greeted Lewis and Clark when they arrived in the Lower Columbia River in 1805.',
    'The tribes helped members of the expedition by bringing them food and helping with navigation.',
    '',
]

In [6]:
print(sari.corpus_sari(src_sents, hyp_sents, refs_sents, legacy=False))

15.549134266001737


In [ ]:
print(bertscore.corpus_bertscore(hyp_sents, refs_sents))

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(-1.0634506940841675, -1.1177774667739868, -1.0874214172363281)


In [46]:
import random
random.seed(42)

indices = random.choices(list(range(0, 256)), k=50)
level = 4
scores = []
with open(f'/srv/scratch6/kew/ats/data/comparative_scores_all_l{level}.jsonl', 'r', encoding='utf8') as f:
    for i, line in enumerate(f):
        if i in indices:
            line = json.loads(line.strip())
            # print('\\toprule')
            print(f'Simp-0 (source) & {line["src_texts"]}' + ' \\\\')
            print(f'Simp-{level} (reference) & {line["tgt_texts"]}' + ' \\\\')
            print(f'MUSS & {line["muss_texts"]}' + ' \\\\')
            # print(line['muss_bertscore'])
            print(f'SUPER & {line["super_texts"]}' + ' \\\\')
            
            print(f'B\_Simp-{level} & {line["fudge_texts"]}' + ' \\\\')
            # print(line['fudge_bertscore'])
            
            # print(line['super_bertscore'])
            print('\\midrule')
            print('\\midrule')
            print()

#     scores.append(line['fudge_bertscore'])
# print(np.array(scores).mean())

Simp-0 (source) & The tribes helped members of the expedition through the winter, bringing them food and assisting with navigation. \\
Simp-4 (reference) & The Native Americans brought the explorers food. They helped the explorers find their way. \\
MUSS & The tribes brought them food during the winter and helped with navigation. \\
Super & The tribes helped members of the expedition through the winter. They brought them food and helped with navigation. \\
B\_Simp-4 & The, they helped members of the expedition. They brought them food and helped them with navigation. \\
\midrule
\midrule

Simp-0 (source) & Changes include moving the earliest date from which tribes must provide documentation of their existence into the 1930s and allowing tribes to reapply for federal recognition under certain circumstances, something previously prohibited. \\
Simp-4 (reference) & The new rules would also let rejected tribes apply again. This was not allowed before. \\
MUSS & Changes include moving the ea